# Libraries

In [2]:
import numpy as np
import pandas as pd

from pkg.evaluation   import Evaluation
from pkg.algorithims  import *
from pkg.neighborhood import *

from time import time


# Simulates Annealing

In [3]:
seeds = np.array([7054, 1354, 23503, 11268, 58283])

ev = Evaluation()

granularity = 2 # estudiado en local search


A continuación se muestran los parámetros que quedan por ajustar y que ajustaremos en las siguientes celdas

In [4]:
def run_simulated_an(seeds, L, k_max, mu, fi, granularity, ev):
    costs       = np.array([])
    evaluations = np.array([])

    ini = time()

    greedy_cost = 574.5513854177129
    t0 = mu * greedy_cost / (- np.log(fi))
    tf = t0 / (1 + k_max)

    for s in seeds:

        # set evaluation calls counter to 0
        ev.total_calls = 0

        np.random.seed(s)

        solution, cost = simulated_an(
            t0         = t0,
            tf         = tf,
            L          = L, 
            slots      = granularity, 
            evaluation = ev
        )

        costs = np.append(costs, cost)
        evaluations = np.append(evaluations, ev.total_calls)


    fin = time()

    df = pd.DataFrame({
        "L"                 : [L],
        "k_max"             : [k_max],
        "mu"                : [mu],
        "fi"                : [fi],
        "tiempo_ejecucion"  : [fin-ini],
        "media_coste"       : [costs.mean()],
        "std_coste"         : [costs.std()],
        "media_evaluaciones": [evaluations.mean()],
        "std_evaluaciones"  : [evaluations.std()],
        "coste_mejor"       : [costs.min()]
    })

    return df


In [13]:
# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.1

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.1

df = run_simulated_an(
    seeds       = seeds,
    L           = L,
    k_max       = k_max,
    mu          = mu,
    fi          = fi,
    granularity = granularity,
    ev          = ev
)


In [16]:
df

,L,k_max,mu,fi,tiempo_ejecucion,media_coste,std_coste,media_evaluaciones,std_evaluaciones,coste_mejor
0,20,80,0.1,0.1,401.022399,387.177988,12.95838,1641.0,0.0,375.85566


## Ajuste de parametros

Ajuste de mu y fi. Guardamos todos los mu y fi que cumplan la condición en un dataframe 

In [19]:
greedy_solution = np.array([ 7,  9, 18,  8, 11, 18, 11, 12,  8, 13, 13, 24, 11, 18, 20, 19])

n = Neighborhood(greedy_solution,2)
candidata = n.get_neighbor()

cost_actual = ev.evaluate(greedy_solution)
cost_cand = ev.evaluate(candidata)
dif = cost_cand - cost_actual

while dif < 0:
    candidata = n.get_neighbor()
    cost_actual = ev.evaluate(greedy_solution)
    cost_cand = ev.evaluate(candidata)

    dif = cost_cand - cost_actual

mus = np.array([])
fis = np.array([])
t = 0


for mu in np.arange(0.1,0.32, 0.02):
    for fi in np.arange(0.1,0.32, 0.02):

        aceptadas = 0
        t +=1

        T = mu * cost_actual / (- np.log(fi))

        for it in range(1000):

            if np.random.rand() < np.exp(-dif/T):
                aceptadas += 1
        
        if aceptadas > 600 and aceptadas < 700: # entre el 80 y el 85 % 
            mus = np.append(mus,mu)
            fis = np.append(fis,fi)

df_mu_fi = pd.DataFrame({
    "mu": mus,
    "fi": fis
})

t

121

In [20]:
df_mu_fi


,mu,fi
0,0.10,0.24
1,0.10,0.26
2,0.10,0.28
3,0.10,0.30
4,0.12,0.18
5,0.12,0.20
6,0.12,0.22
7,0.12,0.24
8,0.12,0.26
9,0.12,0.28


In [21]:
mu = 0.12
fi = 0.24

Ajuste del numero de iteraciones (k_max) y numero de vecinos generados

In [22]:
k = np.array([70, 80, 90])
l = np.array([20, 30, 40])
mu = 0.12
fi = 0.24

df = pd.DataFrame({
        "L"                 : [0],
        "k_max"             : [0],
        "mu"                : [0],
        "fi"                : [0],
        "tiempo_ejecucion"  : [0],
        "media_coste"       : [0],
        "std_coste"         : [0],
        "media_evaluaciones": [0],
        "std_evaluaciones"  : [0],
        "coste_mejor"       : [0]
    })

for k_max in k:
    for L in l:
        df_aux = run_simulated_an(
            seeds       = seeds,
            L           = L,
            k_max       = k_max,
            mu          = mu,
            fi          = fi,
            granularity = granularity,
            ev          = ev
        )

        df = pd.concat( [df, df_aux] )

        df.to_excel('./01_stats/simulated_an' + str(k_max) +'_' + str(L)+  '_v2.xlsx')

        print(f'{k_max} y {L} hecho')

70 y 20 hecho
70 y 30 hecho
70 y 40 hecho
80 y 20 hecho
80 y 30 hecho
80 y 40 hecho
90 y 20 hecho
90 y 30 hecho
90 y 40 hecho


In [23]:
df

,L,k_max,mu,fi,tiempo_ejecucion,media_coste,std_coste,media_evaluaciones,std_evaluaciones,coste_mejor
0,0,0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000
0,20,70,0.12,0.24,372.760833,389.092245,14.157867,1441.0,0.0,378.175002
0,30,70,0.12,0.24,549.618594,385.952360,14.149533,2161.0,0.0,373.058203
0,40,70,0.12,0.24,727.308786,386.099265,13.479597,2881.0,0.0,374.955498
0,20,80,0.12,0.24,404.779764,389.084239,14.144064,1641.0,0.0,378.175002
0,30,80,0.12,0.24,622.266252,385.760519,14.034271,2461.0,0.0,373.058203
0,40,80,0.12,0.24,831.333552,385.962802,13.378865,3281.0,0.0,374.955498
0,20,90,0.12,0.24,463.467776,388.015206,14.999516,1841.0,0.0,375.057124
0,30,90,0.12,0.24,696.764617,385.714727,14.066140,2761.0,0.0,373.058203
0,40,90,0.12,0.24,949.846626,385.959909,13.373734,3681.0,0.0,374.955498


Observando el dataframe, escogemos los parámetros L = 30 y k_max = 70
Los costes medios de todas las combinaciones son muy parecidos, al igual que las desviaciones tipicas.
Alrededor de las 2200 llamadas a la funcion evaluacion se obtienen los mejores resultados.